# Ataques ao RSA

O criptosistema RSA baseia-se na dificuldade em determinar **p** e **q** dado um inteiro N, tal que $N=p*q$ com $p$ e $q$ primos.



## Inversão da chave pública

## Inversão do criptograma